In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

from src.grid_partitioning import Grid

### Main code

In [ ]:
# Read the files needed in input
path_users_classification = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.classified.parquet'
path_users_cells_mapping = './mapping_users_cells_grid_50m.pkl'
# path_grid = './grid_50m.pkl'

# grid = Grid.load_from_pickle(path_grid)
# display(grid.get_grid())

users_labels = pd.read_parquet(path_users_classification)
display(users_labels)

mapping_users_cells = pd.DataFrame(pd.read_pickle(path_users_cells_mapping))
display(mapping_users_cells)

mapping_users_cells['label'] = users_labels['label']
display(mapping_users_cells)
display(mapping_users_cells.info())

In [ ]:
stats_config = {'list_users' : pd.NamedAgg(column='uid', aggfunc=set),
                'num_users' : pd.NamedAgg(column='uid', aggfunc='nunique'),
                'positive_rate' : pd.NamedAgg(column='label', aggfunc='mean')}
prova = (mapping_users_cells.reset_index()
                            .groupby('cell_id')
                            .agg(**stats_config))
prova['negative_rate'] = 1 - prova['positive_rate']


# Sort the cells by number of users.
prova.sort_values(by='num_users', ascending=False, inplace=True)
display(prova)

In [ ]:
# Versione Python non ottimizzata intersezione liste ID utenti celle.
from itertools import combinations

pairs = list(zip(prova.index, prova['list_users']))
counter = {}
cnt_threshold = 10
for (cell_id, list_users), (other_cell_id, other_list_users) in combinations(pairs, 2):
    
    cnt = len(list_users & other_list_users) # Compute the number of users we have doing the set intersection for (cell_id, other_cell_id)

    # Add to the dictionary only the cell pairs that have at least 'threshold' users in common.
    # The threshold should be calculated according to the statistical power we want to have in the hypotesis tests.
    if cnt > cnt_threshold : counter[(cell_id, other_cell_id)] = cnt

In [ ]:
testa = pd.DataFrame.from_dict(counter, orient='index').sort_values(by=0, ascending=False)
del counter
testa